# Drift validation

In [1]:
import iris
import numpy as np
import matplotlib.pyplot as plt
import glob

import pandas as pd

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

from iris.experimental.equalise_cubes import equalise_attributes
import cf_units

In [2]:
output_notebook()

Loading BokehJS ...

### Read data

In [3]:
institution = 'NOAA-GFDL'
model = 'GFDL-CM3'
control_location = '/g/data/ua6'  # /g/data/r87/dbi599
historical_location = '/g/data/ua6'

In [4]:
control_files = glob.glob('%s/drstree/CMIP5/GCM/%s/%s/piControl/mon/ocean/thetao/r1i1p1/thetao_Omon_%s_piControl_r1i1p1_*.nc' %(control_location, institution, model, model) )
historical_files = glob.glob('%s/drstree/CMIP5/GCM/%s/%s/historical/mon/ocean/thetao/r1i1p1/thetao_Omon_%s_historical_r1i1p1_*.nc' %(historical_location, institution, model, model) )
dedrifted_historical_files = glob.glob('/g/data/r87/dbi599/drstree/CMIP5/GCM/%s/%s/historical/mon/ocean/thetao/r1i1p1/dedrifted/thetao_Omon_%s_historical_r1i1p1_*.nc' %(institution, model, model) )
coefficient_file = '/g/data/r87/dbi599/drstree/CMIP5/GCM/%s/%s/piControl/mon/ocean/thetao/r1i1p1/thetao-coefficients_Omon_%s_piControl_r1i1p1_all.nc' %(institution, model, model)

In [5]:
# Set constraints
#min_lat, max_lat = [-62.1, -60.9]
#lat_subset = lambda cell: min_lat <= cell <= max_lat
lev_constraint = iris.Constraint(depth=15)
lat_constraint = iris.Constraint(grid_latitude=-63.5)
lon_constraint = iris.Constraint(grid_longitude=70.5)

In [6]:
def read_files(file_list):
    """Read a list of files into an iris Cube"""
    
    cube = iris.load(file_list, 'sea_water_potential_temperature' & lev_constraint & lat_constraint & lon_constraint)
    equalise_attributes(cube)
    cube = cube.concatenate_cube()
    
    return cube

In [7]:
control_cube = read_files(control_files)
historical_cube = read_files(historical_files)
dedrifted_historical_cube = read_files(dedrifted_historical_files)
coefficient_cube = iris.load_cube(coefficient_file, 'Sea Water Potential Temperature' & lev_constraint & lat_constraint & lon_constraint)

/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:1139: UserWarning: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warnings.warn(msg)


In [8]:
print coefficient_cube.data

[  2.74396027e+02  -1.92017706e-05   3.36148942e-10  -1.17815200e-15]


### Validate polynomial contruction

In [9]:
def apply_polynomial(x, a, b, c, d):
    """Evaluate cubic polynomial.

    The axis argument is not used but is required for the function to be 
      used with numpy.apply_over_axes 

    """

    result = a + b * x + c * x**2 + d * x**3 
    
    return result 

In [10]:
coeff_a, coeff_b, coeff_c, coeff_d = coefficient_cube.data
my_poly = apply_polynomial(control_cube.coord('time').points,
                           coeff_a, coeff_b, coeff_c, coeff_d)

### Plot

In [11]:
iris.FUTURE.cell_datetime_objects = True

In [12]:
print coefficient_cube.attributes['time_unit'], coefficient_cube.attributes['time_calendar']
print historical_cube.coord('time').units, historical_cube.coord('time').attributes['calendar_type']
print control_cube.coord('time').units
print historical_cube.attributes['branch_time']

days since 0001-01-01 00:00:00 noleap
days since 1860-01-01 00:00:00 noleap
days since 0001-01-01 00:00:00
0.0


In [13]:
 historical_cube.coord('time').points[0:5]

array([  15.5,   45. ,   74.5,  105. ,  135.5])

In [14]:
print control_cube.coord('time').points[0:5]

[  15.5   45.    74.5  105.   135.5]


In [15]:
control_time_coord = control_cube.coord('time')
new_unit = cf_units.Unit(historical_cube.coord('time').units, calendar=historical_cube.coord('time').attributes['calendar_type'])  
control_time_coord.convert_units(new_unit)

In [16]:
print control_cube.coord('time').points[0:5]
print control_cube.coord('time').units

[-678519.5 -678490.  -678460.5 -678430.  -678399.5]
days since 1860-01-01 00:00:00


In [17]:
branch_time = float(historical_cube.attributes['branch_time'])
control_start = coefficient_cube.attributes['time_start']

adjustment = control_cube.coord('time').points[0] - branch_time - control_start

control_cube.coord('time').points = control_cube.coord('time').points - adjustment

#assert control_cube.coord('time').points[0] == historical_cube.attributes['time_start'] + branch_time

In [18]:
print control_cube.coord('time').points[0:5]

[  15.5   45.    74.5  105.   135.5]


In [19]:
historical_times = historical_cube.coord('time')
historical_dates = [cell.point.strftime("%Y-%m-%d") for cell in historical_times.cells()]  # cell.point.strftime("%Y-%m-%d")

historical_df = pd.DataFrame({ 'Date' : historical_dates,
                               'historical' : historical_cube.data,
                             })

print type(historical_dates[0])
print historical_dates[1:20]

<type 'str'>
['1860-02-15', '1860-03-16', '1860-04-16', '1860-05-16', '1860-06-16', '1860-07-16', '1860-08-16', '1860-09-16', '1860-10-16', '1860-11-16', '1860-12-16', '1861-01-16', '1861-02-15', '1861-03-16', '1861-04-16', '1861-05-16', '1861-06-16', '1861-07-16', '1861-08-16']


In [26]:
control_times = control_cube.coord('time')
control_dates = [cell.point.strftime("%Y-%m-%d") for cell in control_times.cells()]  # cell.point.strftime("%Y-%m-%d")

control_df = pd.DataFrame({ 'Date' : control_dates,
                            'piControl' : control_cube.data,
                            'drift' : my_poly
                         })

print type(control_dates[0])
print control_dates[1:20]

<type 'str'>
['1860-02-15', '1860-03-16', '1860-04-16', '1860-05-16', '1860-06-16', '1860-07-16', '1860-08-16', '1860-09-16', '1860-10-16', '1860-11-16', '1860-12-16', '1861-01-16', '1861-02-15', '1861-03-16', '1861-04-16', '1861-05-16', '1861-06-16', '1861-07-16', '1861-08-16']


In [27]:
#dataframe_list = [control_df, historical_df]
#dataframe = pandas.concat(dataframe_list, join='inner', axis=1)
df = pd.merge(control_df, historical_df, on='Date', how='outer')

In [28]:
print df

            Date       drift   piControl  historical
0     1860-01-16  274.395729  277.267181  277.017914
1     1860-02-15  274.395163  277.780609  277.968292
2     1860-03-16  274.394598  276.544861  276.023468
3     1860-04-16  274.394014  274.931061  274.006256
4     1860-05-16  274.393431  273.254761  272.680542
5     1860-06-16  274.392848  271.898895  271.812683
6     1860-07-16  274.392266  271.388977  271.489227
7     1860-08-16  274.391676  271.343842  271.350464
8     1860-09-16  274.391095  271.371338  271.369385
9     1860-10-16  274.390515  271.479309  271.422882
10    1860-11-16  274.389935  272.044556  271.783813
11    1860-12-16  274.389357  273.985901  273.956360
12    1861-01-16  274.388769  276.094849  276.829346
13    1861-02-15  274.388210  277.184448  277.718292
14    1861-03-16  274.387652  275.823578  275.768372
15    1861-04-16  274.387076  274.268372  273.717010
16    1861-05-16  274.386500  272.865234  272.278595
17    1861-06-16  274.385925  271.742249  271.

In [29]:
def datetime(x):
    return np.array(x, dtype=np.datetime64)

p1 = figure(x_axis_type="datetime", title="Sea water potential temperature")
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Temperature'

p1.line(datetime(df['Date']), df['piControl'], color='blue', legend='piControl')
p1.line(datetime(df['Date']), df['historical'], color='red', legend='historical')
p1.line(datetime(df['Date']), df['drift'], color='black', legend='drift')
p1.legend.location = "top_left"

#output_file("drift.html", title="Drift Check")

show(p1)  # open a browser

In [24]:
len(my_poly)

6000

In [25]:
len(df)

6000

### TODO

* Spatial plots of drift coefficients for a given level  
* annual mean

In [30]:
control_cube_runmean = control_cube.rolling_window('time', iris.analysis.MEAN, 12)

/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/cube.py:3381: UserWarning: The bounds of coordinate u'time' were ignored in the rolling window operation.
  'the rolling window operation.' % coord_.name())


In [31]:
control_runmean_times = control_cube_runmean.coord('time')
control_runmean_dates = [cell.point.strftime("%Y-%m-%d") for cell in control_runmean_times.cells()]  # cell.point.strftime("%Y-%m-%d")

In [32]:
print control_runmean_dates[1:20]

['1860-08-01', '1860-08-31', '1860-09-30', '1860-10-30', '1860-11-30', '1860-12-30', '1861-01-30', '1861-03-02', '1861-04-01', '1861-05-02', '1861-06-01', '1861-07-02', '1861-08-01', '1861-08-31', '1861-09-30', '1861-10-30', '1861-11-30', '1861-12-30', '1862-01-30']
